## Load modules and read in data

First, we will import all the modules needed for this analysis.

In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
from glob import glob
import os
import random
from sklearn.preprocessing import StandardScaler
from mixed_sigmoid_normalisation import MixedSigmoidScaler,mixed_sigmoid_func

# Set seed to 127
random.seed(127)

%load_ext rpy2.ipython

In [2]:
%%R 

suppressPackageStartupMessages({
    library(broom)
    library(circlize)
    library(ComplexHeatmap)
    library(cowplot)
    library(dendextend)
    library(FactoMineR)
    library(GGally)
    library(ggseg)
    library(glue)
    library(grid)
    library(patchwork)
    library(see)
    library(tidyverse)
})

# Set cowplot theme
theme_set(theme_cowplot())

In [16]:
# Define data directory
infotheory_measure_dir = "infotheory_measures/"

# Load SPI groupings
infotheory_measure_info = pd.read_csv("infotheory_measure_info.csv")

# Find .csv files in the data directory
all_subjects_info_theory_res = pd.concat([pd.read_csv(f) for f in glob(infotheory_measure_dir + "*.csv")]).merge(infotheory_measure_info, on="Measure", how='left')

In [18]:
all_subjects_info_theory_res.head()

,Measure,region_from,region_to,Measure_Type,value,Base_Region,Sample_ID,Measure_name,Group,Group_number
0,entropy_kozachenko,lateraloccipital,lateraloccipital,Univariate,1.420351,lateraloccipital,176542,Entropy,Univariate order-independent,1
1,AIS_kraskov,lateraloccipital,lateraloccipital,Univariate,0.554316,lateraloccipital,176542,Active information storage,Univariate order-dependent,4
2,entropy_kozachenko,parsopercularis,parsopercularis,Univariate,1.385257,parsopercularis,176542,Entropy,Univariate order-independent,1
3,AIS_kraskov,parsopercularis,parsopercularis,Univariate,0.240965,parsopercularis,176542,Active information storage,Univariate order-dependent,4
4,je_kozachenko,lateraloccipital,parsopercularis,Bivariate,2.833943,NaN,176542,Joint entropy,"Bivariate order-independent, undirected",2


In [26]:
%%R -i all_subjects_info_theory_res -o mean_measure_value_by_region

# Generate default color palette
gg_color_hue <- function(n) {
      hues = seq(15, 375, length = n + 1)
      hcl(h = hues, l = 65, c = 100)[1:n]
      }
group_colors <- gg_color_hue(6)

# Let's plot this in the brain
mean_measure_value_by_region <- all_subjects_info_theory_res %>%
  group_by(region_to, Measure_name, Group_number) %>%
  summarise(mean_value = mean(value, na.rm=T)) %>%
  ungroup() %>%
  mutate(label = glue("lh_{region_to}"),
         mean_value = ifelse(mean_value<0, NA_real_, mean_value))
  
mean_measure_value_by_region_dk <- mean_measure_value_by_region %>%
  left_join(., as_tibble(dk))

plot_list <- list()
for (measure in unique(mean_measure_value_by_region_dk$Measure_name)) {
      this_measure_data <- subset(mean_measure_value_by_region_dk, Measure_name==measure)
      measure_group_color <- group_colors[this_measure_data$Group_number][1]
      p <- mean_measure_value_by_region_dk %>%
            filter(Measure_name==measure) %>%
            ggseg(atlas = dk, 
                  # mapping = aes(fill = mean_value_norm),
                  mapping=aes(fill=mean_value),
                  position = "stacked", colour = "gray40", hemisphere="left") +
            theme_void() +
            labs(fill = measure) +
            theme(plot.title = element_blank(),
                  legend.key.width = unit(0.45, "cm"),
                  legend.key.height = unit(0.35, "cm"),
                  legend.position = "bottom") +
            scale_fill_gradient(low="white", high=measure_group_color, na.value="white")
      plot_list[[measure]] <- p
}

wrap_plots(plot_list)
ggsave("../SPIs/figure_drafting/info_theory_measures_in_brain_100_subjects.svg", width=10, height=5, units="in", dpi=300)

`summarise()` has grouped output by 'region_to', 'Measure_name'. You can
override using the `.groups` argument.
Joining with `by = join_by(label)`


merging atlas and data by 'label', 'atlas', 'type', 'hemi', 'side', 'region', 'roi'
merging atlas and data by 'label', 'atlas', 'type', 'hemi', 'side', 'region', 'roi'
merging atlas and data by 'label', 'atlas', 'type', 'hemi', 'side', 'region', 'roi'
merging atlas and data by 'label', 'atlas', 'type', 'hemi', 'side', 'region', 'roi'
merging atlas and data by 'label', 'atlas', 'type', 'hemi', 'side', 'region', 'roi'
merging atlas and data by 'label', 'atlas', 'type', 'hemi', 'side', 'region', 'roi'
merging atlas and data by 'label', 'atlas', 'type', 'hemi', 'side', 'region', 'roi'
merging atlas and data by 'label', 'atlas', 'type', 'hemi', 'side', 'region', 'roi'
merging atlas and data by 'label', 'atlas', 'type', 'hemi', 'side', 'region', 'roi'
merging atlas and data by 'label', 'atlas', 'type', 'hemi', 'side', 'region', 'roi'
merging atlas and data by 'label', 'atlas', 'type', 'hemi', 'side', 'region', 'roi'
In addition: Warning message:
In left_join(., as_tibble(dk)) :
  Detected an

In [37]:
# Load this subject's time series
# z-score each column
subject_TS_data = pd.DataFrame(StandardScaler().fit_transform(np.load(f"{data_dir}/{subject_ID}.npy").T))
subject_TS_data.columns = brain_region_lookup.Brain_Region.values

subject_TS_data['Timepoint'] = subject_TS_data.index + 1

# Extract the source region time series
source_region_TS = subject_TS_data[[f"ctx-lh-{source_base_region}", "Timepoint"]].rename(columns={f"ctx-lh-{source_base_region}": "BOLD_Signal"})

# Melt the data
subject_TS_data_melted = subject_TS_data.melt(id_vars="Timepoint", var_name="Brain_Region", value_name="BOLD_Signal")

# Find the min and max value per measure, keeping the brain region
min_max_region_agg = (this_subject_infotheory_results
                      .groupby("Measure_name")['value_norm']
                      .agg(['min', 'max'])
                      .reset_index()
                      .melt(id_vars="Measure_name", var_name="variable", value_name="value_norm")
                      .merge(this_subject_infotheory_results, on=["Measure_name", "value_norm"], how="left")
                      .drop(columns=["Base_Region"])
                      .rename(columns={"region_to": "Base_Region"})
                    )

min_max_TS_data = subject_TS_data_melted.merge(brain_region_lookup, on="Brain_Region").merge(min_max_region_agg, on="Base_Region").query("Hemisphere=='Left'")

In [ ]:
%%R -i source_region_TS

source_region_TS %>% 
    ggplot(data=., mapping=aes(x=Timepoint, y=BOLD_Signal)) +
    geom_line(linewidth=0.4, color="#be7edf") +
    theme_void() +
    theme(legend.position="none") 

# ggsave("../SPIs/figure_drafting/left_lateral_occipital_TS.svg", width=4, height=1, units="in", dpi=300)

In [ ]:
%%R -i min_max_TS_data 

min_max_TS_data %>% 
    ggplot(data=., mapping=aes(x=Timepoint, y=BOLD_Signal, color=as.factor(Group_number))) +
    geom_line(linewidth=0.4) +
    facet_grid(Measure_name~variable) +
    theme_void() +
    theme(legend.position="none") 
# ggsave("../SPIs/figure_drafting/info_theory_measures_min_max_TS.svg", width=6, height=12, units="in", dpi=300)

In [9]:
# Shut down the JVM at the end of session
jpype.shutdownJVM() 

## How are the measures correlated?

In [40]:
measure_wise_corrs = mean_measure_value_by_region.pivot(index="region_to", columns="Measure_name", values="mean_value_norm").corr(method="spearman")

measure_wise_corrs_bivar =  mean_measure_value_by_region.query("Measure_name not in ['Active information storage', 'Entropy']").pivot(index="region_to", columns="Measure_name", values="mean_value_norm").corr(method="spearman")

In [ ]:
%%R -i measure_wise_corrs,measure_wise_corrs_bivar

# svg("../SPIs/figure_drafting/infotheory_measure_corrs_across_brain_bivar.svg", width=7, height=5, bg=NA)
ht1 <- ComplexHeatmap::Heatmap(measure_wise_corrs_bivar,
                                clustering_method_rows = "ward.D2",
                                clustering_method_columns = "ward.D2",
                                row_names_side = "right",
                                row_dend_side = "left", 
                                row_dend_width = unit(1, "cm"),
                                row_dend_gp = gpar(lwd=unit(0.5, "cm")),
                                row_split = 4,
                                column_split = 3,
                                row_title = NULL,
                                column_title = NULL,
                                show_row_names = TRUE,
                                show_column_names = FALSE,
                                name = "Spearman corr",
                                show_column_dend = FALSE,
                                heatmap_legend_param = list(legend_direction = "horizontal",
                                                            legend_width = unit(5, "cm"))) 

draw(ht1, heatmap_legend_side = "bottom",
    padding = unit(c(2, 2, 2, 2), "mm"),
    background = "transparent")
# dev.off()

### Plot entropy histogram in the brain

In [ ]:
this_subject_infotheory_results.head()

In [ ]:
# Find the min and max value_norm per Measure_name, keeping the region_from and region_to columns

entropy_min_max = (this_subject_infotheory_results
                      .query("Measure_name == 'Entropy'")
                      .groupby("Measure_name")['value_norm']
                      .agg(['min', 'max'])
                      .reset_index()
                      .melt(id_vars="Measure_name", var_name="variable", value_name="value_norm")
                      .merge(this_subject_infotheory_results, on=["Measure_name", "value_norm"], how="left")
                      .sort_values(['Measure_name', 'variable'])
)
entropy_min_region = entropy_min_max.query("variable == 'min'").region_from.values[0]
entropy_max_region = entropy_min_max.query("variable == 'max'").region_from.values[0]
entropy_min_TS = subject_TS_data_melted.query(f"Brain_Region == 'ctx-lh-{entropy_min_region}'").assign(Entropy = "min")
entropy_max_TS = subject_TS_data_melted.query(f"Brain_Region == 'ctx-lh-{entropy_max_region}'").assign(Entropy = "max")

entropy_min_max_df = pd.concat([entropy_min_TS, entropy_max_TS])

entropy_min_max_df.head()

In [58]:
%%R -i entropy_min_max_df

entropy_min_max_df %>%
    group_by(Entropy) %>%
    mutate(mean_x = mean(BOLD_Signal)) %>% 
    ggplot(data=., mapping=aes(x=BOLD_Signal, fill=Entropy)) +
    # geom_histogram(position='identity', alpha=0.7, bins=50) +
    geom_density(position='identity', alpha=0.7) +
    scale_y_continuous(expand=c(0,0)) +
    facet_wrap(Entropy ~ ., nrow=1, scales='fixed') +
    scale_fill_manual(values=c("min"="#FFEFED", "max" = "#F9766D")) +
    xlab("BOLD signal") +
    geom_vline(aes(xintercept=mean_x), linetype="dashed", size=0.5) +
    theme(legend.position='none',
          strip.text = element_blank(),
          strip.background = element_blank())
ggsave('../SPIs/figure_drafting/entropy_min_max_TS_density.svg', width=6, height=2, units="in", dpi=300)

entropy_min_TS